### Imports

In [ ]:
import json
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from pyteal import compileTeal, Mode
from contract import approval_program, clear_state_program
from deployment_functions import *
from helper_functions import *

### Generate Account

In [ ]:
def generate_algorand_keypair():
    private_key, address = account.generate_account()
    print("My address: {}".format(address))
    print("My private key: {}".format(private_key))
    print("My passphrase: {}".format(mnemonic.from_private_key(private_key)))    

### Credentials

In [ ]:
# Opening JSON file
f = open('credentials.json')
 
# returns JSON object as a dictionary
data = json.load(f)
 
creator_address = data["creator_address"]
creator_mnemonic = data["creator_mnemonic"]
creator_private_key = mnemonic.to_private_key(creator_mnemonic)
app_id = data["app_id"]

# Closing file
f.close()

### Initialize Algod Client

In [ ]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# initialize an algodClient
algod_client = algod.AlgodClient(algod_token, algod_address)

### Compiling smart contract and declaring schema

In [ ]:
# get PyTeal approval program
approval_program_ast = approval_program()
# compile program to TEAL assembly
approval_program_teal = compileTeal(approval_program_ast, mode=Mode.Application, version=6)
# compile program to binary
approval_program_compiled = compile_program(algod_client, approval_program_teal)


# get PyTeal clear state program
clear_state_program_ast = clear_state_program()
# compile program to TEAL assembly
clear_state_program_teal = compileTeal(clear_state_program_ast, mode=Mode.Application, version=6)
# compile program to binary
clear_state_program_compiled = compile_program(algod_client, clear_state_program_teal)


# declare application state storage (immutable)
local_ints = 0
local_bytes = 0
global_ints = 0
global_bytes = 0
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)

### Create the Smart Contract

In [ ]:
def create_app_wrapper():
    # create new application
    app_id = create_app(
        algod_client,
        creator_private_key,
        approval_program_compiled,
        clear_state_program_compiled,
        global_schema,
        local_schema,
    )

    with open("credentials.json", "r+") as jsonFile:
        data = json.load(jsonFile)

        data['app_id'] = app_id

        jsonFile.seek(0)  # bring cursor to the top
        json.dump(data, jsonFile, indent = 6)
        jsonFile.truncate()



In [ ]:
if not is_opted_in_app(algod_client, creator_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, creator_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)
